In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas

import warnings
warnings.filterwarnings("ignore")

In [ ]:
path = Path("Resources/cleaned_post_code_registration.csv")

In [ ]:
read0 = pd.read_csv(path)

In [ ]:
read0.head()

In [ ]:
read0.count()

In [ ]:
read0.dtypes


In [ ]:
read0["Fuel Type"].value_counts()

In [ ]:
read0["State"].value_counts()

In [ ]:
read0["Fuel Type"] = read0["Fuel Type"].replace({"BEV":"Battery Electric Vehicle",
                       'ICE':"Internal Combustion Engine",
                       'HFCEV':'Hydrogen Cell Vehicle',
                       'Hybrid/PHEV':'Hybrid'})

In [ ]:
read0.head()

In [ ]:
read1= read0.groupby(["Postcode","Fuel Type"])
read1

In [ ]:
read0.duplicated()

In [ ]:
#Total registrations for each fuel type
types= read0.groupby("Fuel Type").sum()
types= types.drop(['Postcode','State'],axis=1)
types

In [ ]:
#EV & Hybrid registration trends combined

BEV_data= read0.loc[(read0["Fuel Type"]=="Battery Electric Vehicle")|(read0["Fuel Type"]=="Hybrid"),:]

first_count= BEV_data["Registrations as at 31 January 2021"].sum()
second_count= BEV_data["Registrations as at 31 January 2022"].sum()
third_count= BEV_data["Registrations as at 31 January 2023"].sum()

Total_reg= pd.DataFrame({"2021":[first_count],"2022":second_count,"2023":third_count}).T
Total_reg.plot(marker="o", title= "EV ownership trend 2021-23", grid=True, legend=False)
plt.xlabel("Years")
plt.xticks(rotation= 0)
plt.ylabel("No. of Registrations")
plt.savefig("Resources/BEV_ownership.png")
plt.show()

In [ ]:
#Comparison of ICE & EV by percentage

firstyear_total= types["Registrations as at 31 January 2021"].sum()
secondyear_total= types["Registrations as at 31 January 2022"].sum()
thirdyear_total= types["Registrations as at 31 January 2023"].sum()

types["2021"]=types["Registrations as at 31 January 2021"]/firstyear_total*100
types["2022"]=types["Registrations as at 31 January 2022"]/secondyear_total*100
types["2023"]=types["Registrations as at 31 January 2023"]/thirdyear_total*100

types_new= types.drop(['Registrations as at 31 January 2021','Registrations as at 31 January 2022',
                       'Registrations as at 31 January 2023'],axis=1)

types_new= types_new[["2021","2022","2023"]].T
types_new["Hybrid & BEV"]= types_new["Hybrid"]+types_new["Battery Electric Vehicle"]

comparison_df= types_new[["Internal Combustion Engine","Hybrid & BEV"]]
comparison_df.plot(marker="o", title= "EV & ICE ownership trend 2021-23", grid=True, legend=False)
plt.xlabel("Years")
plt.xticks(rotation= 0)
plt.ylabel("No. of Registrations")
plt.savefig("Resources/Ownership trends comparison")

plt.show()
comparison_df

In [ ]:
#EV & combustion engine comparison for 2023
years_df= comparison_df[["Internal Combustion Engine","Hybrid & BEV"]].T
types_2023 = years_df["2023"]
plt.title("EV & Combustion engine ownership 2023")
labels= ["Combustion", "EV"]
plt.pie(types_2023, explode=(0,0.4), labels=labels, colors=("lightskyblue","green"), autopct="%1.1f%%",
          startangle=360)
plt.axis("equal")
plt.tight_layout()
plt.savefig("Resources/Ownership comparison 2023")

plt.show()

In [ ]:
#EV & combustion engine comparison for 2021
types_2023 = years_df["2021"]
plt.title("EV & Combustion engine ownership 2021")
labels= ["Combustion ", "EV"]
plt.pie(types_2023, explode=(0,0.4), labels=labels, colors=("lightskyblue","green"), autopct="%1.1f%%",
          startangle=360)
plt.axis("equal")
plt.savefig("Resources/Ownership comparison 2021")

plt.show()

In [ ]:
#State BEV registrations comparison 

State_BEV= BEV_data.groupby("State").sum()
State_totals= State_BEV[["Registrations as at 31 January 2021","Registrations as at 31 January 2022",
                         "Registrations as at 31 January 2023"]]
renamed= State_totals.rename(columns={"Registrations as at 31 January 2021":"2021",
                           "Registrations as at 31 January 2022":"2022",
                           "Registrations as at 31 January 2023":"2023"})

renamed.plot(kind= "bar", title="BEV registrations by States 2021-23")
plt.xlabel("States")
plt.xticks(rotation= 45)
plt.ylabel("No. of BEV Registrations")
plt.tight_layout()
plt.savefig("Resources/States_comparison")
plt.show()

renamed

In [ ]:
#plotting postcodes with high EV ownership
postcode= pd.read_csv("australian_postcodes.csv")
postcodes_df= postcode.rename(columns={"postcode":"Postcode"})
postcodes_df["Postcode"]= postcodes_df["Postcode"].drop_duplicates()

merged= pd.merge(BEV_data,postcodes_df, on="Postcode", how="inner")

merged_short= merged[["Postcode","Lat_precise","Long_precise","Registrations as at 31 January 2023"]]
BEV_postcodes = merged_short.sort_values(by="Registrations as at 31 January 2023", ascending=False)
top_postcodes= BEV_postcodes.loc[BEV_postcodes["Registrations as at 31 January 2023"]>1000]

BEV_plot = top_postcodes.hvplot.points("Long_precise","Lat_precise",
    geo = True, color="Postcode", size= "Registrations as at 31 January 2023",
    tiles = "OSM", frame_width = 700, frame_height = 500, scale= 0.5, colorbar=False)

BEV_plot